In [1]:
# imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2

# from statsmodels.graphics.tsaplots import plot_acf
# from statsmodels.tsa.stattools import adfuller
# from scipy.ndimage.interpolation import shift

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPClassifier
from sklearn import metrics

import statsmodels.api as sm
from statsmodels.formula.api import ols

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report 
from sklearn.ensemble import GradientBoostingClassifier
%matplotlib inline

In [2]:
# Versões dos pacotes usados no notebook

%reload_ext watermark
%watermark -a "César S. Hooper/ Dezembro 2022" --iversions
!python --version

Author: César S. Hooper/ Dezembro 2022

cv2        : 4.5.4-dev
pandas     : 1.2.4
sklearn    : 0.24.1
statsmodels: 0.12.2
matplotlib : 3.3.4
numpy      : 1.19.5

Python 3.8.8


In [97]:
datos = pd.read_excel('ES time frame 30 min 2022 actualizado.xlsx', sheet_name='Hoja2')
datos = datos.drop(['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2'], axis=1)

In [98]:
datos = datos.fillna(0)
datos.head(10)
columnas = datos.columns[1:9]
columnas

Index(['LSOI Verde', 'LSOI Rojo', 'LSOI L', 'LSOI  S', 'LSOI  I', 'LSOI  O',
       'CCI  Verde', 'CCI Rojo'],
      dtype='object')

In [102]:
k = 1
lista1 = []
for col in columnas:
    lista1.append(datos[str(col)].sum()/k)
    k += 1
lista1

[3528.0, 3552.0, 2693.0, 2577.0, 1199.0, 755.0, 3756.0, 3426.0]

In [103]:
# reemplaza por 1 las opciones coloreadas
k = 1
for col in columnas:
    datos[str(col)] = datos[str(col)].replace(k, 1)
    k += 1

In [109]:
k = 1
lista2 = []
for col in columnas:
    lista2.append(datos[str(col)].sum())
    k += 1
if(lista2 == lista1): print('Listas Iguales')

Listas Iguales


In [143]:
diff = datos['CLOSE'].diff().fillna(0)
datos['diff'] = diff
diff

0        0.00
1        0.25
2        2.25
3        3.75
4        0.50
        ...  
7219   -21.50
7220     3.75
7221     2.00
7222    -3.75
7223     6.75
Name: CLOSE, Length: 7224, dtype: float64

In [177]:
diff2 = []
for ind in diff.index:
    if (diff[ind]>0): 
        diff[ind] = 1
    elif (diff[ind]<0):
        diff[ind] = -1
    else:
        diff[ind] = 0



In [181]:
datos['diff'].rolling(3)

Rolling [window=3,center=False,axis=0]